In [1]:
import lakefs_spec
import pandas as pd

In [3]:
df = pd.read_parquet("lakefs://pydata-hb/main/lakes.parquet")
df

,Hylak_id,Lake_name,Country,Depth_m
0,1,Caspian Sea,Russia,1025.000000
1,2,Great Bear,Canada,446.000000
2,3,Great Slave,Canada,614.000000
3,4,Winnipeg,Canada,36.000000
4,5,Superior,United States of America,406.000000
...,...,...,...,...
99995,99996,,Canada,6.580317
99996,99997,,Canada,9.083196
99997,99998,,Canada,20.822592
99998,99999,,Canada,10.418225


In [6]:
#Error uploading from pandas with branch protection enabled. Weird error message. 
df = pd.read_csv("developer-stats.csv").to_csv("lakefs://pydata-hb/main/developer-stats.csv")
df

#Need to manually commit this.

In [7]:
#Should we put the `put` method in the docs as well. It is in our Quickstart example but not in the api reference (because its inherited from AbstractFileSystem)?
fs = lakefs_spec.LakeFSFileSystem()

fs.put_file("developer-stats.csv", "pydata-hb/main/developer-stats.csv")

In [8]:
# This does not work
from sklearn.model_selection import train_test_split
with fs.transaction as tx:
    branch = tx.create_branch(repository="pydata-hn", source="main",name="demo-experiment")
    train, test = train_test_split(df)
    train.to_csv(f"lakefs://pydata-hb/{branch}/train.csv")
    test.to_csv(f"lakefs://pydata-hb/{branch}/test.csv")
    tx.commit(repository="pydata-hb", branch=branch, message="Create train test split")

NotFoundException: code: 404, reason: Not Found, body: {'message': 'repository not found'}

In [9]:
# This also does not work
from sklearn.model_selection import train_test_split

with fs.transaction as tx:
    branch = tx.create_branch(repository="pydata-hn", source="main",name="demo-experiment")
    train, test = train_test_split(df)
    fs.put(train, f"pydata-hb/{branch}/train.csv")
    fs.put(test, f"pydata-hb/{branch}/test.csv")
    tx.commit(repository="pydata-hb", branch=branch, message="Create train test split")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/maxmynter/Desktop/appliedAI/pydata-hb/Country'

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df)
train.to_csv("train.csv")
test.to_csv("test.csv")

with fs.transaction as tx:
    branch = tx.create_branch(repository="pydata-hn", source="main",name="demo-experiment")
    fs.put("train.csv", f"pydata-hb/{branch}/train.csv")
    fs.put("test.csv", f"pydata-hb/{branch}/test.csv")
    tx.commit(repository="pydata-hb", branch=branch, message="Create train test split")

In [12]:
#Can we set and infer the repostitory (and into branch) from the transaction somehow?
# For merge the name "source_ref" is weird. To me that implies I could also just merge a ref to a specific file. That throws an error. 
with fs.transaction as tx:
    tx.merge(repository="pydata-hb", source_ref="main", into=branch)
    tx.commit(repository="pydata-hb", branch=branch, message="Add lakes data")

No changes to commit on branch 'demo-experiment'.
No changes to commit on branch 'demo-experiment'.


In [ ]:
SHA = "18f815d2f45cf0e0d7ec8b8186c4fc04eaa26d9246b31f91029bec6dd8222f16"
df = pd.read_csv(f"lakefs://pydata-hb/{SHA}")
df